<a href="https://colab.research.google.com/github/yuliiabosher/Adam_Smith_project/blob/main/Congressional_record.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [2]:
import requests
import os
from google.colab import output
import pprint
from io import BytesIO
import PyPDF2
import re
import datetime
import dask

In [3]:
key = input('enter API key ')
os.environ['Congress_API_Key'] = key
output.clear()

In [4]:
def list_endpoints(year,month,day):
    date_today = datetime.datetime(year, month, day)
    endpoints = []
    for i in range(1, 365):
        past_date = date_today - datetime.timedelta(days=i)
        year = past_date.strftime("%Y")
        month = past_date.strftime("%m")
        day = past_date.strftime("%d")
        endpoint = f"https://api.congress.gov/v3/congressional-record/?y={year}&m={month}&d={day}&api_key={key}"
        endpoints.append(endpoint)
    return endpoints

In [5]:
def find_pdf_links(endpoints_list):
  try:
    links_to_pdfs_with_matches = []
    for i in endpoints_list:
        response = requests.get(i)
        if response.json()['Results']['Issues'] != []:
          link_to_pdf_file = response.json()['Results']['Issues'][0]['Links']['FullRecord']['PDF'][0]['Url']
          pdf_file = requests.get(link_to_pdf_file)
          if pdf_file.content[-7:] == b'\n%%EOF\n':
            pdf_data = BytesIO(pdf_file.content)
            pdf_reader = PyPDF2.PdfReader(pdf_data)
            number_of_pages = len(pdf_reader.pages)
            for j in range(number_of_pages):
              page = pdf_reader.pages[j]
              text = page.extract_text()
              adam_smith_matches = re.findall(r'Adam\sSmith\W*', text)
              if adam_smith_matches != []:
                print(link_to_pdf_file)
                links_to_pdfs_with_matches.append(link_to_pdf_file)
    return links_to_pdfs_with_matches
  except Exception as e:
    print(e)
    return links_to_pdfs_with_matches

delayed_pdf_links = dask.delayed(dask.delayed(find_pdf_links)(list_endpoints(2010,5,20)), dask.delayed(find_pdf_links)(list_endpoints(2010,5,20)), \
dask.delayed(find_pdf_links)(list_endpoints(2009,5,20)), dask.delayed(find_pdf_links)(list_endpoints(2008,5,20)), \
dask.delayed(find_pdf_links)(list_endpoints(2007,5,20)))

In [6]:
actual_pdf_links = delayed_pdf_links.compute()
actual_pdf_links

https://www.congress.gov/111/crec/2010/05/12/CREC-2010-05-12.pdf
https://www.congress.gov/111/crec/2010/05/12/CREC-2010-05-12.pdf
https://www.congress.gov/111/crec/2010/04/20/156/56/CREC-2010-04-20.pdf
https://www.congress.gov/111/crec/2010/04/20/156/56/CREC-2010-04-20.pdf
https://www.congress.gov/111/crec/2010/04/16/156/54/CREC-2010-04-16.pdf
https://www.congress.gov/111/crec/2010/03/04/156/30/CREC-2010-03-04.pdf
https://www.congress.gov/111/crec/2010/01/21/CREC-2010-01-21.pdf
https://www.congress.gov/111/crec/2009/11/19/CREC-2009-11-19.pdf
https://www.congress.gov/111/crec/2009/11/19/CREC-2009-11-19.pdf
https://www.congress.gov/111/crec/2009/11/17/CREC-2009-11-17.pdf
https://www.congress.gov/111/crec/2009/11/10/CREC-2009-11-10.pdf
https://www.congress.gov/111/crec/2009/11/07/CREC-2009-11-07.pdf
https://www.congress.gov/111/crec/2009/10/13/CREC-2009-10-13.pdf
https://www.congress.gov/111/crec/2009/09/25/CREC-2009-09-25.pdf
https://www.congress.gov/111/crec/2009/09/16/CREC-2009-09-16.p

['https://www.congress.gov/111/crec/2010/05/12/CREC-2010-05-12.pdf',
 'https://www.congress.gov/111/crec/2010/05/12/CREC-2010-05-12.pdf',
 'https://www.congress.gov/111/crec/2010/04/20/156/56/CREC-2010-04-20.pdf',
 'https://www.congress.gov/111/crec/2010/04/20/156/56/CREC-2010-04-20.pdf',
 'https://www.congress.gov/111/crec/2010/04/16/156/54/CREC-2010-04-16.pdf',
 'https://www.congress.gov/111/crec/2010/03/04/156/30/CREC-2010-03-04.pdf',
 'https://www.congress.gov/111/crec/2010/01/21/CREC-2010-01-21.pdf',
 'https://www.congress.gov/111/crec/2009/11/19/CREC-2009-11-19.pdf',
 'https://www.congress.gov/111/crec/2009/11/19/CREC-2009-11-19.pdf',
 'https://www.congress.gov/111/crec/2009/11/17/CREC-2009-11-17.pdf',
 'https://www.congress.gov/111/crec/2009/11/10/CREC-2009-11-10.pdf',
 'https://www.congress.gov/111/crec/2009/11/07/CREC-2009-11-07.pdf',
 'https://www.congress.gov/111/crec/2009/10/13/CREC-2009-10-13.pdf',
 'https://www.congress.gov/111/crec/2009/09/25/CREC-2009-09-25.pdf',
 'http